# Instalacion de Python

## Configuración de Proyecto

### Creando Entorno Virtual

```sh
virtualenv lazaro
cd lazaro
source bin/activate
```

### Configuracion de Versionamiento

```sh
sudo apt-get install git
```

Iniciando el versionamiento

```sh
git init
git remote add origin git@github.com:Marcelo1180/lazaro.git
```

Creando el documento **.gitignore**

```sh
nvim .gitignore
```

Editando el archivo **.gitignore**

```git
*.pyc
lazaro/settings.py
control_personal/migrations/
```

### Instalación de Dependencias

```sh
pip install django
```

### Creación de Proyecto

```sh
django-admin startproject lazaro
cd lazaro
django-admin startapp control_personal
```

Registrando la aplicación **control_personal** en **lazaro/settings.py**

```python
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',

    'control_personal',
]
```

### Configurando otro modelo para el usuario

```sh
mkdir control_personal/models
touch control_personal/models/__init__.py
touch control_personal/models/usuario.py
```

Editando el archivo **control_personal/models/usuario.py**

```python
from __future__ import unicode_literals

from django.db import models
from django.contrib.auth.models import AbstractUser

# Create your models here.
class UsuarioManager(models.Manager):
    def nombreCompleto(self):
        full_name = '%s %s' % (self.first_name, self.last_name)
        return full_name.strip()

class Usuario(AbstractUser):
    bio = models.TextField(max_length=500, blank=True)
    location = models.CharField(max_length=30, blank=True)
    birth_date = models.DateField(null=True, blank=True)
    objects = UsuarioManager()

    class Meta:
        verbose_name = 'Usuario'
        verbose_name_plural = 'Usuarios'

    # def __str__(self):
    #     return self.objects.nombreCompleto()

    # def __unicode__(self):
    #     return self.objects.nombreCompleto()

```

Agregando al archivo **lazaro/settings.py**

```python
AUTH_USER_MODEL = 'control_personal.Usuario'
```

Editando el archivo **control_personal/models/\__init\__.py**

```python
from usuario import *
```

Editando el archivo **control_personal/admin.py**

```python
from django.contrib import admin
from control_personal.models import Usuario

admin.site.register(Usuario)

```

### Ejecutando las migraciones

```sh
python manage.py makemigrations
python manage.py migrate
```

Creando superusuario para el admin

```sh
python manage.py createsuperuser
```

### Iniciando el servidor (Desarrollo)

```sh
python manage.py runserver
```

### Configuración de Django Rest Framework

```sh
pip install djangorestframework
pip install markdown
pip install django-filter
pip install djangorestframework-jwt
pip install django-cors-headers
```

Registrando la aplicación **rest_framework** en **lazaro/settings.py**

```python
INSTALLED_APPS = [
    ...
    'corsheaders',
    'rest_framework',
]
```

Permisos por base de datos en el Modelo

Creando el archivo **lazaro/permissions.py**

```python
from rest_framework.permissions import DjangoModelPermissions
from rest_framework import permissions

# Permisos globales para asignacion dinamica por base de datos en la tabla grupos y Meta.permission en los modelos
class DjangoGuardianModelPermissions(DjangoModelPermissions):
    perms_map = {
        'GET': ['%(app_label)s.view_%(model_name)s'],
        'OPTIONS': ['%(app_label)s.options_%(model_name)s'],
        'HEAD': [],
        'POST': ['%(app_label)s.add_%(model_name)s'],
        'PUT': ['%(app_label)s.change_%(model_name)s'],
        'PATCH': ['%(app_label)s.change_%(model_name)s'],
        'DELETE': ['%(app_label)s.delete_%(model_name)s'],
    }
    
class HasGroupPermission(permissions.BasePermission):
    def has_permission(self, request, view):
        user_groups = request.user.groups.values_list('name', flat=True)
        required_groups = getattr(view, 'required_groups', {})
        # Return True si alguno de los grupos del usuario estan en required_groups
        if request.user.is_superuser:
            return True
        return set(required_groups).intersection(user_groups)
```

### Personalizando la respuesta JWT para el usuario

```sh
mkdir control_personal/views
touch control_personal/views/__init__.py
touch control_personal/views/jwt_usuario.py
```

Editando el archivo **control_personal/views/jwt_usuario.py**

```python
def jwt_usuario_response(token, user=None, request=None):
    return {
        'token': token,
        'user': 'Respuesta personalizada',
    }
```

Editando el archivo **control_personal/views/\__init\__.py**

```python
from jwt_usuario import *
```

Agregando las direcciones de autenticación de la aplicación en **lazaro/urls.py**

```python
from django.conf.urls import include
from rest_framework_jwt.views import obtain_jwt_token

urlpatterns = [
    ...
    url(r'^api-auth/', include('rest_framework.urls', namespace='rest_framework')),
    url(r'^api-token-auth/', obtain_jwt_token),
]
```

Configurando **rest_framework, cors-headers, jwt-token** en **lazaro/settings.py**

```python
INSTALLED_APPS = [
    ...
]
# RestFramework Configurations --------------------------------------------------------
REST_FRAMEWORK = {
    'DEFAULT_PERMISSION_CLASSES': [
        'lazaro.permissions.DjangoGuardianModelPermissions',
    ],
    'DEFAULT_PAGINATION_CLASS': 'rest_framework.pagination.PageNumberPagination',
    'PAGE_SIZE': 15,
    # JWT authentication Django Rest djangorestframework-jwt
    'DEFAULT_AUTHENTICATION_CLASSES': (
        'rest_framework_jwt.authentication.JSONWebTokenAuthentication',
        'rest_framework.authentication.SessionAuthentication',
        'rest_framework.authentication.BasicAuthentication',
    ),
    'SEARCH_PARAM' : 'filter',
    'ORDERING_PARAM': 'order',
}
import datetime
JWT_AUTH = {
    'JWT_EXPIRATION_DELTA': datetime.timedelta(seconds=36000),
    'JWT_RESPONSE_PAYLOAD_HANDLER': 'control_personal.views.jwt_usuario_response',
    'JWT_AUTH_HEADER_PREFIX': 'Bearer',
}
CORS_ORIGIN_ALLOW_ALL = True
CORS_ALLOW_HEADERS = (
    'x-requested-with',
    'content-type',
    'accept',
    'origin',
    'authorization'
)
# -------------------------------------------------------------------------------------
MIDDLEWARE = [
    ...
]
```

In [ ]:
dd

### Creando una API REST con DRF

```sh
mkdir control_personal/serializers
touch control_personal/serializers/__init__.py
touch control_personal/serializers/usuario.py
touch control_personal/views/usuario.py
```

Editando el archivo **control_personal/serializers/usuario.py**

```python
#!/usr/bin/python
# -*- coding: utf8 -*-
from rest_framework import serializers
from control_personal.models import Usuario


class UsuarioSerializer(serializers.ModelSerializer):
    class Meta:
	fields = '__all__'
        model = Usuario
```

Editando el archivo **control_personal/serializers/\__init\__.py**

```python
from usuario import *
```

Editando el archivo **control_personal/views/usuario.py**

```python
#!/usr/bin/python
# -*- coding: utf8 -*-
from rest_framework import viewsets, filters
from rest_framework.permissions import IsAuthenticated, AllowAny, IsAdminUser, DjangoModelPermissions
from control_personal.serializers import UsuarioSerializer
from control_personal.models import Usuario


class UsuarioViewSet(viewsets.ModelViewSet):
    """
    ## Usuarios del sistema
    Trabajar con usuarios del sistema, solo tiene acceso el administrador del sistema
    """
    permission_classes = [IsAdminUser]
    queryset = Usuario.objects.all()
    serializer_class = UsuarioSerializer
    filter_backends = (filters.SearchFilter, filters.OrderingFilter,)
    search_fields = ('username',)
    ordering_fields = '__all__'
```

Agregando **usuario** al archivo **control_personal/views/\__init\__.py**

```python
from jwt_usuario import *
from usuario import *
```

Agregando las direcciones de autenticación de la aplicación en **lazaro/urls.py**

```python
from django.conf.urls import url
from django.contrib import admin
from django.conf.urls import include
from rest_framework_jwt.views import obtain_jwt_token
from rest_framework.routers import DefaultRouter
from control_personal import views
# from rest_framework_swagger.views import get_swagger_view


router = DefaultRouter()
router.register(r'usuarios', views.UsuarioViewSet)


# schema_view = get_swagger_view(title='Lazaro API')


urlpatterns = [
    url(r'^admin/', admin.site.urls),
    url(r'^api-auth/', include('rest_framework.urls', namespace='rest_framework')),
    url(r'^api-token-auth/', obtain_jwt_token),
    # -------------------------------------------------------------------------------------------
    # API REST
    # -------------------------------------------------------------------------------------------
    url(r'^api/v2/', include(router.urls)),
    # url(r'^docs/', docs_schema_view),
    # -------------------------------------------------------------------------------------------
]
```

### Instalando DOCS basado en swagger

```sh
pip install django-rest-swagger
```

Registrando la aplicación **swagger** en **lazaro/settings.py**

```python
INSTALLED_APPS = [
    ...
    'rest_framework_swagger',
]
```

Agregando las direcciones de **swagger** en **lazaro/urls.py**

```python
from rest_framework_swagger.views import get_swagger_view


docs_schema_view = get_swagger_view(title='Lazaro API')


urlpatterns = [
    ...
    url(r'^docs/', docs_schema_view),
]
```

Estructura de Django

http://docs.python-guide.org/en/latest/writing/structure/

Tmux sessiones personalizadas

http://minimul.com/increased-developer-productivity-with-tmux-part-4.html

http://stackoverflow.com/questions/5609192/how-to-set-up-tmux-so-that-it-starts-up-with-specified-windows-opened

https://github.com/jimeh/tmuxifier

https://github.com/tmuxinator/tmuxinator